# LAB 1 


@author: Abillelatus (Ryan Herrin)

Code for testing and modeling Powershell Obscuration classification using a provided 
data set with predetermined features. PCA may be the preferred method for the unsupervised approach as 
the data provided is not labled. 

Current Models do not include the "sha1" and "fpath" columns


In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns; sns.set(style='white')

%matplotlib inline

In [2]:
# Define Global variables
# Define Global
original_data_path = "../dataset/all.csv" # this file is originally zipped
# Some columns are empty for future use. Set to False to not include them
include_unused_columns = False 

# Define unused columns here
unused_columns = ['vt_harmless', 'vt_undetected', 'vt_malicious', 'vt_suspicious',
				  'avclass_name', 'is_obf', 'obf_name']

# Remove the string columns sha1 and fpath
include_sha1_fpath = False

# Remove them from the list if set to False 
if include_sha1_fpath == False:
	unused_columns.append('sha1')
	unused_columns.append('fpath')

+ Create a function that imports the dataset and returns a workable formatted version

In [3]:
def create_workable_data(file_path, lst_unused_columns):
	'''import the dataset and format the data into a pandas dataframe.
	+ Read in CSV as dataframe
	+ Remove extra header if there is one 
	+ Remove unused columns if set 
	+ Typecast DataTypes to string and integers
	+ Returns Pandas DataFrame object'''
	try:
		pd_df = pd.read_csv(file_path) # Read in CSV as dataframe
	except FileNotFoundError():
		print("Could not find data file. Make sure the file is unzipped...")
		sys.exit(1)
		
	# Remove Duplicates
	pd_df.drop_duplicates('sha1', inplace=True)
		
	# Check to see if duplicate header is present. Remove if there is 
	if pd_df.at[0, 'sha1'] == 'sha1':
		pd_df = pd_df.iloc[1:] # .iloc[] integer-loc based indexing for selecting by position 
		
	# Remove unused columns if global var "include_unused_columns" is set to False
	if include_unused_columns == False:
		pd_df = pd_df.drop(columns=lst_unused_columns)

	# Convert all to float 
	for col in pd_df.columns.tolist():
		pd_df[col] = pd_df[col].astype(float)
	
	return(pd_df)

In [4]:
# Create a Data Frame from the csv 
data_df = create_workable_data(original_data_path, unused_columns)

C:\Users\Xclusive\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


There are a lot of scripts inlcuded that of zero size or are small one line scripts. We should investigate 
the distribution of script sizes. 

Create reusable function to remove rows from dataframe

In [5]:
def remove_df_rows(data_as_df, char_limit):
    '''Function to remove rows based on the char_limit. char_limit is depicted by
    the doc_char_count feature. 
    @Params:
    - data_as_df (Pandas DataFrame): Data set
    - char_limit (int): Will remove all rows less than or equal to this value
    '''
    working_data = data_as_df # Copy to return
    indx_to_remove = [] # List of index values to remove from DF
    # Find Index's of rows that match the use case 
    for indx in working_data.index.values:
        if float(working_data.at[indx, 'doc_char_count']) <= char_limit:
            indx_to_remove.append(indx)
        else:
            pass

    print('{} row(s) removed that had a document char lenth of {} or less.'.format(
        len(indx_to_remove), char_limit
    ))

    return(working_data.drop(indx_to_remove))

- Remove all rows that have documents with 0 char length

In [6]:
data_df = remove_df_rows(data_df, 0)

1 row(s) removed that had a document char lenth of 0 or less.


Lets create a histogram and show some analytics to see the spread of document length 

In [17]:
# Show distribution in text first because the current data does not translate over to histogram  
def show_freq_list(data, interval, max):
    """Show a list of amount of scripts that fall between the frequency intervals
    from 0 to the user defined interval. The max set the top limit, as in anything 
    above the max value will be included in the '<Max> and Above'.
    """
    max_limit = max
    low_cnt = 0
    high_cnt = interval
    name_list = []
    count_list = []
    percent_list = [] # Percentage of scripts covered
    
    print("Char Count\t|\tNum of Scripts\t|\tPercent of Data")
    print("----------\t|\t--------------\t|\t---------------")

    while low_cnt < max_limit:
        low_cnt = low_cnt + 1
        name_list.append(str(low_cnt) + '-' + str(high_cnt))

        num_of_instances = len(
            data_df[(data_df['doc_char_count'] >= low_cnt) & (data_df['doc_char_count'] <= high_cnt)]
        )

        count_list.append(num_of_instances)
        percent_list.append(num_of_instances / data.shape[0])

        print('{}\t\t\t{}\t\t{}'.format(name_list[-1], count_list[-1], percent_list[-1]))

        # Update values
        low_cnt = low_cnt + interval -1
        high_cnt = high_cnt + interval

    print('above {}\t\t\t{}\t\t{}'.format(
        max_limit, 
        data.shape[0] - sum(count_list), 
        (data.shape[0] - sum(count_list))/data.shape[0])
    )

    #TODO: Create aggregate. Ex. 1-100: %   |   101-max: %   | max-above: %


show_freq_list(data_df, 200, 50000)

Char Count	|	Num of Scripts	|	Percent of Data
----------	|	--------------	|	---------------
1-100			4274		0.020922263559819854
101-200			6446		0.0315547288036029
201-300			7203		0.03526042686508714
301-400			7374		0.036097513217152925
401-500			7323		0.035847855884080675
501-600			7166		0.03507930291756413
601-700			7680		0.03759545721558645
701-800			7661		0.037502447620912475
801-900			6310		0.030888975915410222
901-1000			7240		0.03544155081261014
1001-1100			5181		0.025362247895046014
1101-1200			4925		0.0241090659878598
1201-1300			4339		0.021240454278441356
1301-1400			4006		0.019610338750734285
1401-1500			3740		0.01830820442529861
1501-1600			3650		0.017867632661053456
1601-1700			3420		0.01674172704131584
1701-1800			3216		0.015743097709026824
1801-1900			3049		0.01492559232426082
1901-2000			2964		0.014509496769140396
2001-2100			2759		0.013505972195026434
2101-2200			2626		0.012854905032308597
2201-2300			2444		0.011963971020168397
2301-2400			2376		0.011631094576072058
2401

### EDA - Data Transformation

In [ ]:
# Option to modify the data to turn feature values from count to ratio
data_to_percent = True

def transform_feature_count_to_percent(dataframe):
    """Function to turn certain features from count to percentage (ratio) values. It
    does this be deviding the feature value by the document char count. Document char count
    is also a feature within the dataset and will be ignored along with some other features.
    This transformation will help try to mitigate the affects of scaling, because as the script 
    gets larger than then char count would go up too. 

    Returns transformed Pandas DataFrame 
    """
    working_data = dataframe  # Create DF to return
    char_count_col_name = 'doc_char_count'

    # List of features that would not likely be susceptible to scaling issues
    non_sus_feat_lst = ['doc_char_count', 'doc_avg_line_len', 'doc_min_line_len',
                        'doc_line_count', 'doc_mcomment_count', 'doc_entropy']

    # Start looping through the working data and change the values 
    for row in list(working_data.index.values):
        for col_name in working_data.columns.tolist():
            # Skip columns that we don't want to modify
            if col_name not in non_sus_feat_lst:
                working_data.at[row, col_name] = (
                    int(working_data.at[row, col_name]) / int(working_data.at[row, char_count_col_name])
                    )
            else:
                pass

    return(working_data)

# Transform if data_to_percent is true
if data_to_percent:
    data_df = transform_feature_count_to_percent(data_df)